In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.2 MB/s eta 0:00:00


In [3]:
!pip install -q scikit-llm==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 663.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.7 MB/s eta 0:00:00


In [4]:
# Imports
import json
import os
import skllm
from skllm.config import SKLLMConfig
from skllm import ZeroShotGPTClassifier
from skllm import MultiLabelZeroShotGPTClassifier
from skllm import FewShotGPTClassifier
from skllm import DynamicFewShotGPTClassifier
from skllm.preprocessing import GPTVectorizer
from skllm.preprocessing import GPTSummarizer
from skllm.preprocessing import GPTTranslator

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "version 0" --iversions

Author: version 0

json: 2.0.9



## Definindo o Acesso aos LLMs

O Scikit-LLM fornece uma interface conveniente para acessar os modelos GPT da OpenAI. O uso desses modelos não é gratuito e requer uma chave API.

Embora o custo da API seja relativamente baixo, dependendo do volume de seus dados e da frequência das chamadas à API, esses custos podem aumentar.

Portanto, é importante planejar e gerenciar cuidadosamente seu uso para controlar custos.

Lembre-se sempre de revisar os [detalhes de preços](https://openai.com/pricing/) e os termos de uso do OpenAI antes de começar a usar o Scikit-LLM.

In [6]:
file_json = open('/content/FIAP_aula_DL.json')

keys = json.load(file_json)

In [7]:
# API key e Organization ID
OPENAI_SECRET_KEY = keys['OPENAI_SECRET_KEY']
OPENAI_ORG_ID = keys['OPENAI_ORG_ID']

In [8]:
# Configura os IDs
SKLLMConfig.set_openai_key(OPENAI_SECRET_KEY)
SKLLMConfig.set_openai_org(OPENAI_ORG_ID)

Agora é só escolher o LLM que você deseja:

https://platform.openai.com/docs/models/overview

## Fine Tuning do LLM Para Classificação de Texto Zero-Shot

Um dos recursos do Scikit-LLM é a capacidade de realizar classificação de texto zero-shot. Scikit-LLM oferece duas classes para esta finalidade:

- `ZeroShotGPTClassifier`: Usado para classificação (previsão) de um único Label (por exemplo, para análise de sentimentos em texto).

- `MultiLabelZeroShotGPTClassifier`: Usado para classificação de diversos labels (por exemplo, para categorização de produtos).

Vamos implementar as 2 classes com nossos próprios dados. Começaremos com a Classificação de Texto Zero-Shot, que tem duas abordagens:

- Classificação de Texto Zero-Shot com Dados Rotulados
- Classificação de Texto Zero-Shot com Dados Não Rotulados

Leia o manual em pdf no Capítulo 8 com a definição desta técnica de classificação.

### Classificação de Texto Zero-Shot com Dados Rotulados

In [9]:
# Frases de avaliações de curso (coloque nesta célula seus próprios dados). Esses são dados de entrada.
# São dados históricos para o fine-tuning do modelo
avaliacoes_cursos = [
    "Este curso é simplesmente espetacular, nunca vi nada igual.",
    "Este curso foi absolutamente maravilhoso. O conteúdo era amplo e os projetos eram muito realistas.",
    "Eu realmente adorei o curso! O conteúdo teve algumas reviravoltas inesperadas que me mantiveram envolvido até o fim.",
    "O curso foi bom. Não foi ótimo, mas também não foi ruim. Um curso decente e único.",
    "Não gostei muito do curso. O conteúdo era bastante previsível e faltava profundidade aos projetos.",
    "Este curso não foi do meu agrado. Parecia muito lento e o conteúdo não era envolvente o suficiente.",
    "O curso foi bom. Não foi impressionante nem decepcionante. Foi muito bom.",
    "Fiquei impressionado com o curso! A qualidade foi excelente e as aulas foram de primeira qualidade.",
    "Não gostei nada do curso. O conteúdo era desinteressante e as aulas foram, na melhor das hipóteses, razoáveis.",
    "O curso foi decente. Teve seus momentos, mas não foi consistentemente envolvente."
]

In [10]:
# Labels (coloque nesta célula seus próprios dados). Esses são dados de saída.
# São dados históricos para o fine-tuning do modelo. Cada label deve corresponder a uma frase acima.
labels_avaliacoes_cursos = ["positivo",
                            "positivo",
                            "positivo",
                            "neutro",
                            "negativo",
                            "negativo",
                            "neutro",
                            "positivo",
                            "negativo",
                            "neutro"]

In [11]:
# Novos dados para testar o modelo
novas_avaliacoes = [

    # Avaliação positiva
    "O curso foi fantástico! Fiquei cativado pelas aulas do começo ao fim.",

    # Avaliação negativa
    "Achei o curso muito chato. O conteúdo se movia muito devagar e as aulas foram abaixo da média.",

    # Avaliação neutra
    "O curso foi razoável. Não foi o melhor que já vi, mas certamente não foi o pior."
]

In [12]:
# Cria o classificador a partir do LLM "gpt-3.5-turbo"
modelo_1 = ZeroShotGPTClassifier(openai_model = "gpt-3.5-turbo")

In [13]:
# Treina o modelo (fine-tuning) com seus próprios dados
modelo_1.fit(X = avaliacoes_cursos, y = labels_avaliacoes_cursos)

ZeroShotGPTClassifier()

In [14]:
# Usamos o modelo para fazer previsões com os novos dados
previsoes_labels_novos_dados = modelo_1.predict(X = novas_avaliacoes)

100%|██████████| 3/3 [00:02<00:00,  1.05it/s]


In [15]:
# Loop para extrair o texto da avaliação do curso e a previsão de label do modelo
for avaliacao, sentimento in zip(novas_avaliacoes, previsoes_labels_novos_dados):
    print(f"\nTexto da Avaliação: {avaliacao}\nSentimento Previsto: {sentimento}\n")


Texto da Avaliação: O curso foi fantástico! Fiquei cativado pelas aulas do começo ao fim.
Sentimento Previsto: positivo


Texto da Avaliação: Achei o curso muito chato. O conteúdo se movia muito devagar e as aulas foram abaixo da média.
Sentimento Previsto: negativo


Texto da Avaliação: O curso foi razoável. Não foi o melhor que já vi, mas certamente não foi o pior.
Sentimento Previsto: neutro



### Classificação de Texto Zero-Shot com Dados Não Rotulados

In [16]:
# Criamos o objeto (não é necessário fornecer o modelo a ser usado, a menos que você queira usar outro)
modelo_2 = ZeroShotGPTClassifier()

In [17]:
# Ao treinar o modelo, não passamos dados, somente as classes que o modelo deve considerar para previsão
modelo_2.fit(None, ["positivo", "negativo", "neutro"])

ZeroShotGPTClassifier()

In [18]:
# Novos dados para testar o modelo
novas_avaliacoes = [

    # Avaliação positiva
    "O curso foi espetacular! O instrutor trazia materiais atualizados e mostrava tudo passo a passo.",

    # Avaliação negativa
    "O curso não foi tão bom. Poderia ter sido melhor.",

    # Avaliação positiva
    "Sem dúvidas é um curso de outro nível, nível bem alto. Curso de muita qualidade."
]

In [19]:
# Fazemos as previsões com os novos dados
labels_previstos = modelo_2.predict(novas_avaliacoes)

100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


In [20]:
# Loop para extrair o texto da avaliação do curso e a previsão de label do modelo
for avaliacao, sentimento in zip(novas_avaliacoes, labels_previstos):
    print(f"\nTexto da Avaliação: {avaliacao}\nSentimento Previsto: {sentimento}\n")


Texto da Avaliação: O curso foi espetacular! O instrutor trazia materiais atualizados e mostrava tudo passo a passo.
Sentimento Previsto: positivo


Texto da Avaliação: O curso não foi tão bom. Poderia ter sido melhor.
Sentimento Previsto: negativo


Texto da Avaliação: Sem dúvidas é um curso de outro nível, nível bem alto. Curso de muita qualidade.
Sentimento Previsto: positivo



## Fine Tuning do LLM Para Classificação de Texto Multi-Label Zero-Shot

Leia o manual em pdf no Capítulo 8 com a definição desta técnica de classificação.

In [21]:
# Lista de produtos de uma loja (coloque nesta célula seus próprios dados). Esses são dados de entrada.
lista_produtos = [
    "Relógio analógico com pulseira de couro.",
    "Batedeira elétrica com 5 velocidades.",
    "Jaqueta impermeável em tamanho médio.",
    "Conjunto de panelas antiaderentes.",
    "Smartphone com tela de 6 polegadas e 128GB de armazenamento.",
    "Livro sobre jardinagem para iniciantes.",
    "Tênis esportivo masculino tamanho 41.",
    "Máquina de café expresso automática.",
    "Kit de maquiagem com 20 cores diferentes.",
    "Fone de ouvido sem fio com cancelamento de ruído."
]

In [22]:
# Lista de categorias dos produtos acima (coloque nesta célula seus próprios dados). Esses são dados de saída.
lista_categorias = [
    ["Acessórios", "Moda", "Relógios"],
    ["Eletrodomésticos", "Cozinha", "Eletrônicos"],
    ["Moda", "Roupas", "Casacos"],
    ["Cozinha", "Utensílios", "Panelas"],
    ["Eletrônicos", "Celulares", "Tecnologia"],
    ["Livros", "Jardinagem", "Educação"],
    ["Moda", "Calçados", "Esportes"],
    ["Eletrodomésticos", "Cozinha", "Café"],
    ["Beleza", "Cosméticos", "Maquiagem"],
    ["Eletrônicos", "Acessórios", "Áudio"]
]

In [23]:
# Novos dados
novos_produtos = [
    "Blusa esportiva feminina respirável para corrida.",          # ["Moda", "Esportes"]
    "Laptop ultrafino com tela touch de 15 polegadas.",           # ["Eletrônicos", "Tecnologia"]
    "Perfume masculino com fragrância amadeirada.",               # ["Moda", "Beleza"]
    "Mixer de cozinha com lâminas de aço inoxidável.",            # ["Eletrodomésticos", "Cozinha"]
    "Livro de receitas vegetarianas com fotos ilustrativas.",     # ["Livros", "Cozinha"]
    "Relógio digital esportivo com monitoramento cardíaco.",      # ["Acessórios", "Esportes"]
    "Bolsa feminina em couro com compartimentos internos.",       # ["Moda", "Acessórios"]
    "Caixa de som Bluetooth resistente à água.",                  # ["Eletrônicos", "Áudio"]
    "Apostila de exercícios de guitarra para iniciantes.",        # ["Livros", "Educação"]
    "Panquequeira antiaderente com cabo ergonômico."              # ["Cozinha", "Utensílios"]
]

In [24]:
# Cria o modelo para previsão de no máximo 3 labels
modelo_3 = MultiLabelZeroShotGPTClassifier(max_labels = 3)

In [25]:
# Treina o modelo com nossos dados
modelo_3.fit(X = lista_produtos, y = lista_categorias)

MultiLabelZeroShotGPTClassifier()

In [26]:
# Faz previsões com novos dados
previsoes_labels = modelo_3.predict(X = novos_produtos)

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [27]:
# Loop para imprimir o resultado
for produto, categorias in zip(novos_produtos, previsoes_labels):
    print(f"Descrição do Produto: {produto}\nCategorias Previstas: {categorias}\n")

Descrição do Produto: Blusa esportiva feminina respirável para corrida.
Categorias Previstas: ['Moda', 'Esportes']

Descrição do Produto: Laptop ultrafino com tela touch de 15 polegadas.
Categorias Previstas: ['Eletrônicos', 'Tecnologia']

Descrição do Produto: Perfume masculino com fragrância amadeirada.
Categorias Previstas: ['Beleza']

Descrição do Produto: Mixer de cozinha com lâminas de aço inoxidável.
Categorias Previstas: ['Cozinha', 'Utensílios']

Descrição do Produto: Livro de receitas vegetarianas com fotos ilustrativas.
Categorias Previstas: ['Cozinha', 'Livros']

Descrição do Produto: Relógio digital esportivo com monitoramento cardíaco.
Categorias Previstas: ['Relógios', 'Esportes']

Descrição do Produto: Bolsa feminina em couro com compartimentos internos.
Categorias Previstas: ['Acessórios', 'Moda']

Descrição do Produto: Caixa de som Bluetooth resistente à água.
Categorias Previstas: ['Eletrônicos', 'Áudio']

Descrição do Produto: Apostila de exercícios de guitarra para

## Fine Tuning do LLM Para Classificação de Texto Few-Shot

Com FewShotGPTClassifier é possível realizar uma classificação de poucos disparos, o que significa que as amostras de treinamento serão adicionadas ao prompt e passadas para o modelo.

Embora a API permaneça a mesma do classificador zero shot, há alguns aspectos diferentes:

- O “treinamento” requer alguns dados de treinamento rotulados.

- O conjunto de treinamento deve ser pequeno o suficiente para caber em um único prompt (recomenda-se até 10 amostras por rótulo).

Nota: Como o modelo não está sendo treinado novamente, mas usa os dados de treinamento durante a inferência, pode-se dizer que esta ainda é uma abordagem (diferente) de disparo zero.

Leia o manual em pdf no Capítulo 8 com a definição desta técnica de classificação.

In [28]:
# Define X (entrada)
X = avaliacoes_cursos

In [29]:
# Define y (saída)
y = labels_avaliacoes_cursos

In [30]:
# Cria o modelo
modelo_4 = FewShotGPTClassifier(openai_model = "gpt-3.5-turbo")

In [31]:
# Fine tuning do modelo
modelo_4.fit(X, y)

FewShotGPTClassifier()

In [32]:
# Faz as previsões
labels_previstos = modelo_4.predict(X)

100%|██████████| 10/10 [00:10<00:00,  1.02s/it]


In [33]:
# Loop para imprimir o resultado
for avaliacao, sentimento in zip(novas_avaliacoes, labels_previstos):
    print(f"Texto da Avaliação: {avaliacao}\nSentimento Previsto: {sentimento}\n")

Texto da Avaliação: O curso foi espetacular! O instrutor trazia materiais atualizados e mostrava tudo passo a passo.
Sentimento Previsto: positivo

Texto da Avaliação: O curso não foi tão bom. Poderia ter sido melhor.
Sentimento Previsto: positivo

Texto da Avaliação: Sem dúvidas é um curso de outro nível, nível bem alto. Curso de muita qualidade.
Sentimento Previsto: positivo



## Classificação de Texto Dynamic Few-Shot

Leia o manual em pdf no Capítulo 8 com a definição desta técnica de classificação.

In [34]:
# Define X (entrada)
X = avaliacoes_cursos

In [35]:
# Define y (saída)
y = labels_avaliacoes_cursos

In [36]:
# Cria o modelo
modelo_5 = DynamicFewShotGPTClassifier(n_examples = 3)

In [37]:
# Ajuste fino
modelo_5.fit(X, y)

Building index for class `positivo` ...


100%|██████████| 4/4 [00:01<00:00,  3.60it/s]


Building index for class `neutro` ...


100%|██████████| 3/3 [00:01<00:00,  2.72it/s]


Building index for class `negativo` ...


100%|██████████| 3/3 [00:00<00:00,  4.19it/s]


DynamicFewShotGPTClassifier()

In [38]:
# Previsões
labels_previstos = modelo_5.predict(X)

100%|██████████| 10/10 [00:12<00:00,  1.23s/it]


In [39]:
# Loop para imprimir o resultado
for avaliacao, sentimento in zip(novas_avaliacoes, labels_previstos):
    print(f"Texto da Avaliação: {avaliacao}\nSentimento Previsto: {sentimento}\n")

Texto da Avaliação: O curso foi espetacular! O instrutor trazia materiais atualizados e mostrava tudo passo a passo.
Sentimento Previsto: positivo

Texto da Avaliação: O curso não foi tão bom. Poderia ter sido melhor.
Sentimento Previsto: positivo

Texto da Avaliação: Sem dúvidas é um curso de outro nível, nível bem alto. Curso de muita qualidade.
Sentimento Previsto: positivo



## Usando LLM Para Vetorização de Texto

Scikit-LLM fornece a classe `GPTVectorizer` para converter o texto de entrada em uma representação vetorial de dimensão fixa. Essa tarefa pode ser usada como pré-processamento de dados.

Cada vetor resultante é uma matriz de números flutuantes, que é uma representação da frase correspondente.

Vamos obter uma representação vetorizada das frases abaixo

In [40]:
# Coloque aqui seus próprios dados
frases = ["A IA vai revolucionar as indústrias.",
          "A robótica cria soluções automatizadas.",
          "IoT conecta dispositivos para troca de dados."]

In [41]:
# Cria o vetorizador
vetorizador = GPTVectorizer()

In [42]:
# Extrai os vetores
vetores = vetorizador.fit_transform(frases)

100%|██████████| 3/3 [00:00<00:00,  6.36it/s]


In [43]:
print(vetores)

[[-0.02094314 -0.00779687  0.00561232 ...  0.01025733 -0.00571619
   0.00312589]
 [-0.00839186 -0.00503843  0.01148325 ...  0.01418324  0.00248771
   0.00039845]
 [-0.01801247  0.00736281  0.02174279 ...  0.02358186  0.00089182
  -0.00105649]]


## Usando LLM Para Sumarização de Texto

Os modelos GPT são muito úteis para resumir textos. A biblioteca Scikit-LLM fornece o estimador `GPTSummarizer` para esta tarefa. Vamos ver isso em ação, resumindo as longas análises fornecidas abaixo.

In [44]:
# Coloque aqui seus próprios dados
frases = [
    "Almocei no Bella Trattoria nesta semana e foi uma experiência inesquecível. A atenção ao detalhe foi notável, os pratos eram deliciosos e o espaço era aconchegante. Optei pelo risoto de cogumelos, que estava cremoso e saboroso. A seleção de sobremesas era de dar água na boca. Se você deseja um almoço elegante, este é o lugar ideal.",
    "Esta tarde, parei no The Taco Hub para um lanche e não poderia estar mais satisfeito. Embora seja um estabelecimento de comida rápida, o sabor estava no ponto. Escolhi o taco de carne e estava recheado e apetitoso. Os nachos eram frescos e cobertos com queijo derretido. O atendimento foi eficiente e o pessoal muito cortês. Um excelente local para matar a fome rapidamente.",
    "O Tea Time Terrace é onde adoro relaxar e desfrutar de um chá de qualidade. O clima é sereno e as opções de chá são sempre frescas. Sempre opto pelo chá verde e o scone de cranberry pois são deliciosos."
]

In [45]:
# Cria o sumarizador definindo o tamanho do sumário
sumarizador = GPTSummarizer(openai_model = "gpt-3.5-turbo", max_words = 15, focus = "almoço")

In [46]:
# Sumariza o texto
sumarios = sumarizador.fit_transform(frases)

100%|██████████| 3/3 [00:04<00:00,  1.39s/it]


In [47]:
print(sumarios)

['Almocei no Bella Trattoria, experiência inesquecível, atenção ao detalhe, pratos deliciosos, espaço aconchegante.'
 'The Taco Hub is a great place for a quick lunch with delicious tacos and fresh nachos.'
 'O Tea Time Terrace é um lugar tranquilo para desfrutar de chá de qualidade.']


## Usando LLM Para Tradução de Texto

Os modelos GPT podem ser usados para traduzir texto. Podemos traduzir um texto para um idioma de interesse usando o módulo `GPTTranslator`.

In [48]:
# Cria o objeto tradutor
tradutor = GPTTranslator(openai_model = "gpt-3.5-turbo", output_language = "Portuguese")

In [49]:
# Texto a ser traduzido
texto_para_traduzir = ["Deep learning course at FIAP for LLMs studies."]

In [50]:
# Tradução
texto_traduzido = tradutor.fit_transform(texto_para_traduzir)

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


In [51]:
print(f"Texto Original em Inglês: \n{texto_para_traduzir[0]}\n\nTexto Traduzido em Português: \n{texto_traduzido[0]}")

Texto Original em Inglês: 
Deep learning course at FIAP for LLMs studies.

Texto Traduzido em Português: 
Curso de aprendizado profundo na FIAP para estudos de LLMs.
